<a href="https://colab.research.google.com/github/actuallyachraf/ai-notebooks/blob/main/Fashion_MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms

# calculate train time, writing train data to files etc.
import time
import pandas as pd
import json

In [3]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP,self).__init__()
    self.fc1 = nn.Linear(in_features=28*28,out_features=500)
    self.fc2 = nn.Linear(in_features=500,out_features=200)
    self.fc3 = nn.Linear(in_features=200,out_features=100)
    self.out = nn.Linear(in_features=100,out_features=10)

  def forward(self,t):
    t = t.view(-1,28*28)
    t = self.fc1(t)
    t = F.relu(t)
    # fc2
    t = self.fc2(t)
    t = F.relu(t)
    # fc3
    t = self.fc3(t)
    t = F.relu(t)
    # out
    t = self.out(t)
    return t


In [8]:
def train(net, loader, loss_func, optimizer):
    net.train()
    n_batches = len(loader)
    for inputs, targets in loader:
        inputs = Variable(inputs)
        targets = Variable(targets)

        output = net(inputs)
        loss = loss_func(output, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
         # print statistics
    running_loss = loss.item()
    print('Training loss: %.3f' %( running_loss))

def main():
    train_set = torchvision.datasets.FashionMNIST(
        root = './FMNIST',
        train = True,
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
    )
    mlp = MLP()
    loader = torch.utils.data.DataLoader(train_set, batch_size = 500)
    optimizer = optim.Adam(mlp.parameters(), lr=0.01)
    loss_func=nn.CrossEntropyLoss()
    for i in range(0,15):
        train(mlp,loader,loss_func,optimizer)
    print("Finished Training")
    torch.save(mlp.state_dict(), "./mlpmodel.pt")
    test_set = torchvision.datasets.FashionMNIST(
        root = './FMNIST',
        train = False,
        download = False,
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
    )
    testloader = torch.utils.data.DataLoader(test_set, batch_size=4)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = mlp(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

In [12]:
main()

Training loss: 0.459
Training loss: 0.399
Training loss: 0.373
Training loss: 0.313
Training loss: 0.339
Training loss: 0.278
Training loss: 0.299
Training loss: 0.281
Training loss: 0.265
Training loss: 0.281
Training loss: 0.263
Training loss: 0.242
Training loss: 0.258
Training loss: 0.233
Training loss: 0.222
Finished Training
Accuracy of the network on the 10000 test images: 88 %
